In [1]:
# Importação de bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import inspect
import os
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
# Carregamento do dataset
dataset = pd.read_csv("../data/creditcard.csv", sep=";", encoding="latin1")

# Converte todos os nomes de colunas para minúsculas
dataset.columns = dataset.columns.str.lower()